# Flatfielding Stitcher

If you're acquiring mosaics, you probably want to flatfield the images before you stitch them. Chances are, you probably don't care about the intermediate flatfielded & unstitched images. This utility will flatfield & stitch your images in one step without leaving all the intermediate gunk lying around!

The flatfielder works by first identifying a group of images with a common background profile. The flatfielder forms a group by finding images with a common imaging channel & exposure length. It then randomly selects N images from that group and computes a median image. The median image is then subtracted from each member of the group. The resulting flatfielded images are fed onward into the stitcher, and the final outputs are written to the processed_imgs/stitched folder.

In [ ]:
from ipyfilechooser import FileChooser
from common import ImgMeta
from common.utils import try_load_mfile, extract_meta
from improc.flatfield import background_from_paths, trunc_sub
from improc.stitching import stitch
from collections import defaultdict
from multiprocessing import Pool
import numpy as np
import os
import itertools
import PIL
import pathlib
import random

# num imgs to sample when constructing a background image
SAMPLE_SIZE = 100

# num cores available
N_CORES = 18

EXPERIMENT_DIRECTORY = "/nfs/turbo/umms-sbarmada/experiments"

def key_func(meta: ImgMeta):
    return (meta.time_point, meta.col, meta.row)

def read_and_flatfield(path: pathlib.Path, bg: np.ndarray) -> np.ndarray:
    img = np.array(PIL.Image.open(path)).astype(bg.dtype)
    return trunc_sub(img, bg)

def save(arr: np.ndarray, meta: ImgMeta):
    well_label = meta.path.name.split(".")[0].split("_")[0] + ".tif"
    relative_path = meta.path.relative_to(experiment_base / "raw_imgs").parent
    output_base = experiment_base / "processed_imgs" / "stitched" / relative_path
    os.makedirs(output_base, exist_ok=True)
    print(f"T{meta.time_point}|{well_label}")
    PIL.Image.fromarray(arr).save(output_base / well_label)

def flatfield_and_stitch(args):
    background, metas = args
    sorted_metas = sorted(metas, key=lambda x: x.montage_idx)
    images = [read_and_flatfield(meta.path, background) for meta in sorted_metas]
    stitched = stitch(spec.microscope, images)
    save(stitched, sorted_metas[0])
    
def verify_experiment_dir(chooser):
    path = pathlib.Path(fc.selected_path)
    try:
        global experiment_base
        global spec
        spec = try_load_mfile(path)
        experiment_base = path
        chooser.title = ""
    except Exception as e:
        chooser.title = f'<b style="color:red">Error: {e}</b>'

fc = FileChooser(EXPERIMENT_DIRECTORY)
fc.title = "<b>Select experiment directory</b>"
fc.register_callback(verify_experiment_dir)
display(fc)

In [ ]:
# group acquisitions by (channel, exposure) for flatfielding
acquisition_groups = defaultdict(list)
for well in spec.wells:
    for exposure in well.exposures:
        raw_img_glob = (experiment_base / "raw_imgs").glob(f"{exposure.channel}/**/{well.label}*.tif")
        acquisition_groups[(exposure.channel, exposure.exposure_ms)] += map(extract_meta, raw_img_glob)
    

with Pool(N_CORES) as p:
    for params, group in acquisition_groups.items():

        # generate backgrounds from grouped acquisitions
        print(f"Flatfielding and stitching for {params}...")
        sample = map(lambda x: x.path, random.choices(group, k=min(SAMPLE_SIZE, len(group))))
        background = background_from_paths(sample).astype(np.uint16)

        # now further subdivide the acquisition groups into montage sets and order by montage idx
        metas_sets = [(background, list(metas)) for _, metas in itertools.groupby(group, key_func)]
        for _ in p.map(flatfield_and_stitch, metas_sets):
            continue

print("Done!")